In [31]:
import requests
import json
import pandas as pd
import re

## Chargement du json principal et nettoyage des données

In [2]:
url = "https://www.open-medicaments.fr/api/v1/medicaments?limit=100&query=paracetamol"
json = requests.get(url).json()

df=pd.DataFrame(json)

reg=r'([\D]*)(\d+)(.*),(.*)'

df_clean = df['denomination'].str.extract(reg)

df_clean = df_clean.rename(index=str, columns={0: "nom_generique",
                                    1: "dosage",
                                    2: "unite",
                                    3: "type"})


df_clean['mul']=1000
df_clean['mul']=df_clean['mul'].where(df_clean['unite'].str.strip()=="g",1)

df_clean.head()

,nom_generique,dosage,unite,type,mul
0,PARACETAMOL ZYDUS,500,mg,gélule,1
1,PARACETAMOL RATIOPHARM,1000,mg,comprimé,1
2,PARACETAMOL EG,1,g,comprimé,1000
3,PARACETAMOL SANDOZ,500,mg,gélule,1
4,PARACETAMOL RATIOPHARM,500,mg,gélule,1


In [3]:
df_clean['dosage'] = df_clean['dosage'].fillna(0).astype(int)*df_clean['mul']
df_clean.head()

,nom_generique,dosage,unite,type,mul
0,PARACETAMOL ZYDUS,500,mg,gélule,1
1,PARACETAMOL RATIOPHARM,1000,mg,comprimé,1
2,PARACETAMOL EG,1000,g,comprimé,1000
3,PARACETAMOL SANDOZ,500,mg,gélule,1
4,PARACETAMOL RATIOPHARM,500,mg,gélule,1


## Chargement des fiches médicaments

In [ ]:
pages_ICS = [f"https://www.open-medicaments.fr/api/v1/medicaments/{elm['codeCIS']}" for elm in json]

json_ICS=[requests.get(url).json() for url in pages_ICS]

In [32]:
reg = r'(\d+)'
libelles = [medicament["presentations"][0]["libelle"] for medicament in json_ICS]

nb_types = pd.DataFrame({"gelules":[re.findall(reg1,libelle)[-1] for libelle in libelles]})
nb_types.head()

,gelules
0,16
1,8
2,8
3,16
4,16
